# Proyecto 2
## Ciencias de Datos en Python

- Diego Morales - 13001893
- Jerry Rivera - 24005123

In [6]:
import boto3
import pandas as pd
import numpy as np
import configparser
import psycopg2

ModuleNotFoundError: No module named 'sql_queries_2'

## Inicialización

In [3]:
config = configparser.ConfigParser()
config.read('config.cfg')

aws_rds_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                           aws_secret_access_key=config.get('IAM', 'SECRET_KEY'),
                           region_name='us-east-1')

try:
    response = aws_rds_conn.create_db_instance(
                            DBInstanceIdentifier=config.get('SAKILA', 'DB_INSTANCE_ID'),
                            DBName=config.get('SAKILA', 'DB_NAME'),
                            MasterUsername=config.get('SAKILA', 'DB_USERNAME'),
                            MasterUserPassword=config.get('SAKILA', 'DB_PASSWORD'),
                            Port=int(config.get('SAKILA', 'DB_PORT')),
                            DBInstanceClass='db.t3.micro',
                            Engine='postgres',
                            PubliclyAccessible=True,
                            AllocatedStorage=20,
                            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')]
                            )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print('La instancia ya existe')
except Exception as ex:
    print('Error', ex)


{'DBInstance': {'DBInstanceIdentifier': 'sakila-db-pg', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'postgres', 'DBInstanceStatus': 'creating', 'MasterUsername': 'admin_postgres', 'DBName': 'sakila', 'AllocatedStorage': 20, 'PreferredBackupWindow': '07:11-07:41', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0b4d5bb09a6396352', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres16', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-069af3dd3aa36b4d8', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-02d2d39959fbcddb6', 'SubnetAvailabilityZone': {'Name': 'us-east-1b'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-032422906c9f500fb', 'SubnetAvailabilityZone': {'Name': 'us-east-1f'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'sub

## Obtención de instancia

In [5]:
try:
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=config.get('SAKILA', 'DB_INSTANCE_ID'))
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)
except Exception as ex:
    print('Error', ex)

sakila-db-pg.clcwc6cec8sd.us-east-1.rds.amazonaws.com


## Conexión a base de datos

In [11]:
import ddl as query

try:
    db_pg_conn = psycopg2.connect(
                    database=config.get('SAKILA', 'DB_NAME'),
                    user=config.get('SAKILA', 'DB_USERNAME'),
                    password=config.get('SAKILA', 'DB_PASSWORD'),
                    port=config.get('SAKILA', 'DB_PORT'),
                    host=RDS_HOSTNAME
    )
    cursor = db_pg_conn.cursor()
    cursor.execute(query.DDL_QUERY)
    db_pg_conn.commit()
except Exception as ex:
    print('Error', ex)

Error syntax error at or near "EXIST"
LINE 57: CREATE TABLE IF NOT EXIST detalle_ingreso(
                             ^

